In [ ]:
# Import necessary libraries
!pip install folium --user

In [ ]:
import pandas as pd
import folium

In [ ]:
%%bigquery df

select ST_GEOGFROMTEXT(ml_generate_text_llm_result) as geo
, * EXCEPT(ml_generate_text_llm_result)
from
ML.GENERATE_TEXT (
  MODEL`bqml_llm.llm_model`,
  (SELECT
  distinct locations,
   CONCAT('I need coordinates for ',locations,' in San Francisco. Give me just the precise coordinates. Use longitude first. Example input: location "Golden Gate Park", output: POINT(-122.422439 37.826804). Always use POINT(). Use 6 decimal points. Remove all other additional text and characters.') AS prompt
   FROM bigquery-public-data.san_francisco_film_locations.film_locations where locations like "%Bay Bridge%"),
   STRUCT (0.2 AS temperature,
    1 AS top_p,
    20 AS top_k,
    TRUE AS flatten_json_output))
WHERE ml_generate_text_llm_result IS NOT NULL

In [ ]:
# Define a function to extract latitude and longitude from POINT format
def extract_lat_lon(point):
    point = point.replace('POINT', '').replace('(', '').replace(')', '')
    lon, lat = point.split()
    return float(lat), float(lon)

# Create a new column with extracted latitude and longitude
df['latitude'], df['longitude'] = zip(*df['geo'].apply(extract_lat_lon))

In [ ]:
# Create a folium map centered at the first location - SKIP THIS FROM 1+ run TO PLOT MORE ITERATIONS
map = folium.Map(location=[df['latitude'].iloc[0], df['longitude'].iloc[0]], zoom_start=40)

In [ ]:
# Add markers to the map for each location
for index, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['locations']
    ).add_to(map)

# Display the map
map